In [1]:
%load_ext autoreload
%autoreload 2
import time, base64,ssl, os, re,json5, pprint,random , math, hashlib, inspect,requests
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By                   # Locators (ID, CLASS_NAME, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait       # Waits for elements to appear
from selenium.webdriver.support import expected_conditions as EC  # Conditions like "visible", "clickable"
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from app.utils import Helper
from app.operation_executor import OperationExecutor
import pandas as pd
import numpy as np
from io import StringIO
import json
import requests

In [1]:
import time, base64, os, re
import pandas as pd
from selenium import webdriver                                # Controls the browser
from selenium.webdriver.common.by import By                   # Locators (ID, CLASS_NAME, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait       # Waits for elements to appear
from selenium.webdriver.support import expected_conditions as EC  # Conditions like "visible", "clickable"
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin
 
SELECTORS = {
    "filter_list": (By.ID, "filterlist"),
    "last_filter_li": (By.XPATH, "./li[last()]"),
    "fund_links": (By.CSS_SELECTOR, "#fund-perform-table a"),
    "portfolio_button": (By.CLASS_NAME, "portfolio"),
    "show_more": (By.ID, "showMore1"),
    "companies_table": (By.ID, "companies-table"),
    "page_header": (By.TAG_NAME, "h1"),
    "tab_content": (By.CLASS_NAME, "tab-content")
}
 
BASE_URL = r"https://www.iciciprulife.com/fund-performance/all-products-fund-performance-details.html"
PDF_FOLDER = "INSR_PDF"
XLS_FILE = "INSURANCE_MASTER_XLS.xlsx"
BATCH_SIZE = 20
COOLDOWN = 30
 
os.makedirs(PDF_FOLDER, exist_ok=True)
 
 
# --- Utility Functions ---
def clean_text(text):
    text = text.strip()
    text = re.sub(r'[\\/*?:"<>|\/]', "_", text.lower())
    return text
 
def save_pdf(driver,filename):
    pdf = driver.execute_cdp_cmd("Page.printToPDF", {
        "printBackground": True,
        "paperWidth": 8.27,
        "paperHeight": 11.69
    })
   
    save_path = os.path.join(PDF_FOLDER, f"{filename}.pdf")
    with open(save_path, "wb") as f:
        f.write(base64.b64decode(pdf['data']))
 
def save_table_to_excel(driver, content, writer):
    tables = driver.find_elements(By.TAG_NAME, "table")
    all_data = []
 
    for table in tables:
        rows = table.find_elements(By.TAG_NAME, "tr")
        for row in rows:
            cells = row.find_elements(By.XPATH, ".//th | .//td")
            row_data = [cell.text.strip() for cell in cells]
            if any(row_data):
                all_data.append(row_data)
        all_data.append([])
 
    if all_data:
        df = pd.DataFrame(all_data)
        df.insert(0,"url",content["url"])
        df.insert(1,"ins",content["name"])
        df.insert(2,"sfin",content["sfin"])
        df.to_excel(writer, sheet_name=content["name"][:31], index=False, header=False)
         
    # file_path = os.path.join(CONFIG["xls_folder"], filename)
    # df.to_excel(file_path, index=False, header=False)
 
def execute_scripts(driver, element=None, type=""):
    if type == "":
        print("Type not Correct")
        return driver
 
    REQ_SCRIPTS = {
        "color": """
            const style = document.createElement('style');
            style.type = 'text/css';
            style.innerHTML = `
                html, body {
                    background: white !important;
                    color: #111 !important;
                    -webkit-print-color-adjust: exact !important;
                    print-color-adjust: exact !important;
                }
                * {
                    color: #111 !important;
                    background: white !important;
                    border-color: #111 !important;
                }
                a, span, div, td, th, p, h1, h2, h3, h4, h5, h6 {
                    color: #111 !important;
                }
            `;
            document.head.appendChild(style);
        """
    }
 
    driverReturn = driver.execute_script(REQ_SCRIPTS[type], element) if element else driver.execute_script(REQ_SCRIPTS[type])
    time.sleep(1)
    return driverReturn
 
# --- Browser Setup ---
options = Options()
options.add_argument("start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36")
# options.add_argument("--headless")
driver = webdriver.Chrome(options=options)
driver.get(BASE_URL)
 
excelWriter = pd.ExcelWriter(XLS_FILE,engine="xlsxwriter")
 
WebDriverWait(driver, 5).until(EC.presence_of_element_located(SELECTORS["tab_content"]))
 
filter_list = driver.find_element(*SELECTORS["filter_list"])
last_all = filter_list.find_element(*SELECTORS["last_filter_li"])
last_all.click()
time.sleep(2)
 
pdf_links = driver.find_elements(*SELECTORS["fund_links"])
 
total_links = len(pdf_links)
print("Total links:", total_links)
 
for batch_start in range(0, total_links, BATCH_SIZE):
    batch = pdf_links[batch_start:batch_start + BATCH_SIZE]
    print(f"\nProcessing batch {(batch_start // BATCH_SIZE) + 1} with {len(batch)} links")
 
    for j, link in enumerate(batch):
        i = batch_start + j
        try:
            driver.execute_script("window.open(arguments[0].href, '_blank');", link)
            WebDriverWait(driver, 5).until(lambda d: len(d.window_handles) == 2)
            driver.switch_to.window(driver.window_handles[-1])
           
            try:
                performance_element = WebDriverWait(driver, 8).until(
                    EC.element_to_be_clickable(SELECTORS["portfolio_button"])
                )
                performance_element.click()
                WebDriverWait(driver, 8).until(lambda d: len(d.window_handles) == 3)
                driver.switch_to.window(driver.window_handles[-1])
                time.sleep(2)
            except:
               
                print("Portfolio click failed or not present")
           
            try:driver.find_element(*SELECTORS["show_more"]).click()
            except:driver.execute_script("document.getElementById('showMore1').click();")
 
            WebDriverWait(driver, 8).until(
                EC.presence_of_element_located(SELECTORS["companies_table"])
            )
 
            execute_scripts(driver, type="color")
       
            header = driver.find_element(*SELECTORS["page_header"]).text
            file_name, sfin = header.split("\n")
            content = {
                "url":driver.current_url,
                "name": clean_text(file_name),
                "sfin": sfin
            }
           
            print(f"[{i+1}] INSR NAME: {content["name"]} PDF URL: {content["url"]}")
 
            save_pdf(driver,file_name)
            save_table_to_excel(driver,content,excelWriter)
 
        except Exception as e:
            print(f"[FAIL] {file_name}")
            print(f"[{i+1}] Failed: {e}")
        finally:
            while len(driver.window_handles) > 1:
                driver.switch_to.window(driver.window_handles[-1])
                driver.close()
            driver.switch_to.window(driver.window_handles[0])
 
    if batch_start + BATCH_SIZE < total_links:
        print(f"Sleeping {COOLDOWN} seconds before next batch...")
        time.sleep(COOLDOWN)
       
excelWriter.close()


Processing batch 2 with 20 links
[FAIL] Flexi Growth Fund
[21] Failed: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=140.0.7339.208); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x7ff7119c1eb5+80197]
	GetHandleVerifier [0x0x7ff7119c1f10+80288]
	(No symbol) [0x0x7ff7117402fa]
	(No symbol) [0x0x7ff71172c0a5]
	(No symbol) [0x0x7ff71175144a]
	(No symbol) [0x0x7ff7117c7be6]
	(No symbol) [0x0x7ff7117e8132]
	(No symbol) [0x0x7ff7117c0153]
	(No symbol) [0x0x7ff711788b02]
	(No symbol) [0x0x7ff7117898d3]
	GetHandleVerifier [0x0x7ff711c7e83d+2949837]
	GetHandleVerifier [0x0x7ff711c78c6a+2926330]
	GetHandleVerifier [0x0x7ff711c986c7+3055959]
	GetHandleVerifier [0x0x7ff7119dcfee+191102]
	GetHandleVerifier [0x0x7ff7119e50af+224063]
	GetHandleVerifier [0x0x7ff

InvalidSessionIdException: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x7ff7119c1eb5+80197]
	GetHandleVerifier [0x0x7ff7119c1f10+80288]
	(No symbol) [0x0x7ff71174011c]
	(No symbol) [0x0x7ff711787c1d]
	(No symbol) [0x0x7ff7117c0242]
	(No symbol) [0x0x7ff7117bac14]
	(No symbol) [0x0x7ff7117b9cca]
	(No symbol) [0x0x7ff71170a755]
	GetHandleVerifier [0x0x7ff711c7e83d+2949837]
	GetHandleVerifier [0x0x7ff711c78c6a+2926330]
	GetHandleVerifier [0x0x7ff711c986c7+3055959]
	GetHandleVerifier [0x0x7ff7119dcfee+191102]
	GetHandleVerifier [0x0x7ff7119e50af+224063]
	(No symbol) [0x0x7ff711709731]
	GetHandleVerifier [0x0x7ff711dab568+4182008]
	BaseThreadInitThunk [0x0x7ffc3706e8d7+23]
	RtlUserThreadStart [0x0x7ffc378a8d9c+44]


In [ ]:

import requests
 
session = requests.Session()
 
session.headers.update({
    # 'Authorization': 'Bearer YOUR_API_TOKEN',  # if needed
    'Accept': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
})
 
 
from datetime import datetime, timedelta
 
def get_date_range(start_date, end_date):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    delta = (end - start).days + 1
    return [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range(delta)]
 
# Example usage
dates = get_date_range("2019-04-01", "2025-03-31")
# print(dates)
 
 
dates = ['2025-08-15', '2025-08-16', '2025-08-17', '2025-08-18', '2025-08-19', '2025-08-20', '2025-08-21', '2025-08-22', '2025-08-23', '2025-08-24']
 
for idx, value in enumerate(dates[:-1]):
    print(dates[idx],dates[idx+1])
   
    start_date,end_date = dates[idx],dates[idx+1]
    url = f"https://www.amfiindia.com/api/download-nav-history?strMFID=all&FromDate={start_date}&ToDate={start_date}"
 
    # Make the request
    response = session.get(url, verify=False)
 
    # Check if request was successful
    if response.status_code == 200:
        file_name = f"{start_date}.xlsx"
        with open(file_name, 'wb') as f:
            f.write(response.content)
        print("✅ Excel file downloaded successfully.")
    else:
        print(f"❌ Failed to download. Status code: {response.status_code}")
   
    time.sleep(10)
 
import os
import json
import time
import logging
import requests
from datetime import datetime, timedelta
 
 
TODAY = datetime.now().strftime('%Y%m%d')
 
with open("config.json", "r") as f:
    config = json.load(f)
 
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(f"nav_downloader{TODAY}.log"),
        logging.StreamHandler()
    ]
)
 
os.makedirs(config["download_path"], exist_ok=True)
 
 
session = requests.Session()
session.headers.update({
    'Accept': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
})
 
def get_date_range(start_date, end_date):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    delta = (end - start).days + 1
    return [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range(delta)]
 
def download_nav_for_date(date):
    url = f"{config['base_url']}?strMFID=all&FromDate={date}&ToDate={date}"
    headers = config.get("headers",{})
    logging.info(f"Requesting NAV data for {date}")
   
    try:
        response = session.get(url, verify=config["verify_ssl"], headers=headers)
        if response.status_code == 200:
            file_path = os.path.join(config["download_path"], f"{date}.xlsx")
            with open(file_path, 'wb') as f:
                f.write(response.content)
            logging.info(f" Downloaded: {file_path}")
        else:
            logging.warning(f" Failed for {date}. Status code: {response.status_code}")
    except Exception as e:
        logging.error(f"Exception occurred for {date}: {e}")
 
def main():
    dates = get_date_range(config["start_date"], config["end_date"])
    for date in dates:
        download_nav_for_date(date)
        time.sleep(config["sleep_seconds"])
 
if __name__ == "__main__":
    main()
 
{
  "download_path": "./downloads",
  "start_date": "2025-08-15",
  "end_date": "2025-08-17",
  "base_url": "https://www.amfiindia.com/api/download-nav-history",
  "verify_ssl": false,
  "sleep_seconds": 4,
  "headers":{
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept": "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
    "Referer": "https://www.amfiindia.com/"
}
}
 
@echo off
REM Dir
cd /d "C:\Users\kaustubh.keny\Projects\office-work\law-tribunal"
 
REM Program Run
python nav_downloader.py
 
REM Pause to keep the window open
pause
 
hi
 
wfh aahes ?
 
hi 
mi ek code dila toh run karshil ka ?? python cha aahe
 
nhi
 
office madhe ahe
 
ha de na code
 
karto run
 
import requests
import pandas as pd
from io import BytesIO,StringIO
 
session = requests.Session()
session.headers.update({"User-Agent": "Mozilla/5.0"})
 
def fetch_rba():
    url = "https://www.rba.gov.au/statistics/tables/xls/f01d.xlsx"
    r = session.get(url, timeout=15)
    r.raise_for_status()
   
    df = pd.read_excel(BytesIO(r.content))
    df_T = pd.DataFrame(df.iloc[-5:].values, columns=df.iloc[1].values).T
    # df_T = df.iloc[-5:].T.reset_index()
    df_T.columns = df_T.iloc[0]
    df_T.drop(0, inplace=True)
    # df_T = df_T.drop(0).set_index(df_T.columns[0])
    return df_T
 
def fetch_bcra():
    url = "https://www.bcra.gob.ar/PublicacionesEstadisticas/Principales_variables_i.asp"
    r = session.get(url, timeout=15)
    r.raise_for_status()
    return pd.read_html(r.text)[0]
 
def fetch_banxico(token, series_ids, start, end):
   
    #mapper
    series_map = {
        "SF61745": "Target rate 1/",
        "SF331451": "Overnight TIIE funding rate 2/",
        "SF43783": "28 day TIIE 3/",
        "SF43878": "91 day TIIE 3/",
        "SF111916": "182 day TIIE 3/"
    }
   
    #api call
    url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{','.join(series_ids)}/datos/{start}/{end}"
    headers = {"Bmx-Token": token}
    r = session.get(url, headers=headers, timeout=15)
    r.raise_for_status()
    data = r.json()
 
    records = {}
    for s in data['bmx']['series']:
        sid = s['idSerie']
        rec = {d['fecha']: d['dato'] for d in s['datos']}
        records[sid] = rec
    df = pd.DataFrame.from_dict(records, orient="index")
    df.columns = pd.to_datetime(df.columns, dayfirst=True)
   
    df.index = df.index.map(lambda x: series_map.get(x, x))
    # print(df)
   
    return df.sort_index(axis=1)
 
def fetch_boc(series_ids, start, end):
   
    #mapper
    series_map={
        "V39079":"Target for the overnight rate",
        "CL.CDN.MOST.1DL":"Overnight money market financing rate1",
        "V39078":"Bank rate - Daily",
        "V80691310":"Bank rate - Weekly",
        "V122530":"Bank rate - Monthly"
    }
   
    #api call
    url = f"https://www.bankofcanada.ca/valet/observations/{','.join(series_ids)}/json?start_date={start}&end_date={end}"
    r = session.get(url, timeout=15)
    r.raise_for_status()
    data = r.json()
 
    records = {}
    for row in data["observations"]:
        date = row["d"]
        for sid in series_ids:
            records.setdefault(sid, {})[date] = row.get(sid, {}).get("v")
    df = pd.DataFrame.from_dict(records, orient="index")
    df.columns = pd.to_datetime(df.columns)
   
    df.index = df.index.map(lambda x: series_map.get(x, x))
   
    return df.sort_index(axis=1)
 
with pd.ExcelWriter("central_banks.xlsx") as writer:
    token = "1846d489e18513d4306b70e257a21edb56e82325de0139af91dd2176975029bf"
    fetch_banxico(token, ["SF61745","SF331451","SF43783","SF43878","SF111916"], "2025-09-18", "2025-09-24")\
        .to_excel(writer, sheet_name="Banxico")
    fetch_boc(["V39079","CL.CDN.MOST.1DL","V39078","V80691310","V122530"], "2025-09-18", "2025-09-24")\
        .to_excel(writer, sheet_name="BoC")
    # fetch_rba().to_excel(writer, sheet_name="RBA")
    fetch_bcra().to_excel(writer, sheet_name="BCRA", index=False)
 
notebook madhe kar 
 
central_banks.xlsx
 
tyms
 
import time, base64, os, re
import pandas as pd
from selenium import webdriver                                # Controls the browser
from selenium.webdriver.common.by import By                   # Locators (ID, CLASS_NAME, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait       # Waits for elements to appear
from selenium.webdriver.support import expected_conditions as EC  # Conditions like "visible", "clickable"
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin
 
SELECTORS = {
    "filter_list": (By.ID, "filterlist"),
    "last_filter_li": (By.XPATH, "./li[last()]"),
    "fund_links": (By.CSS_SELECTOR, "#fund-perform-table a"),
    "portfolio_button": (By.CLASS_NAME, "portfolio"),
    "show_more": (By.ID, "showMore1"),
    "companies_table": (By.ID, "companies-table"),
    "page_header": (By.TAG_NAME, "h1"),
    "tab_content": (By.CLASS_NAME, "tab-content")
}
 
BASE_URL = r"https://www.iciciprulife.com/fund-performance/all-products-fund-performance-details.html"
PDF_FOLDER = "INSR_PDF"
XLS_FILE = "INSURANCE_MASTER_XLS.xlsx"
BATCH_SIZE = 20
COOLDOWN = 30
 
os.makedirs(PDF_FOLDER, exist_ok=True)
 
 
# --- Utility Functions ---
def clean_text(text):
    text = text.strip()
    text = re.sub(r'[\\/*?:"<>|\/]', "_", text.lower())
    return text
 
def save_pdf(driver,filename):
    pdf = driver.execute_cdp_cmd("Page.printToPDF", {
        "printBackground": True,
        "paperWidth": 8.27,
        "paperHeight": 11.69
    })
   
    save_path = os.path.join(PDF_FOLDER, f"{filename}.pdf")
    with open(save_path, "wb") as f:
        f.write(base64.b64decode(pdf['data']))
 
def save_table_to_excel(driver, content, writer):
    tables = driver.find_elements(By.TAG_NAME, "table")
    all_data = []
 
    for table in tables:
        rows = table.find_elements(By.TAG_NAME, "tr")
        for row in rows:
            cells = row.find_elements(By.XPATH, ".//th | .//td")
            row_data = [cell.text.strip() for cell in cells]
            if any(row_data):
                all_data.append(row_data)
        all_data.append([])
 
    if all_data:
        df = pd.DataFrame(all_data)
        df.insert(0,"url",content["url"])
        df.insert(1,"ins",content["name"])
        df.insert(2,"sfin",content["sfin"])
        df.to_excel(writer, sheet_name=content["name"][:31], index=False, header=False)
         
    # file_path = os.path.join(CONFIG["xls_folder"], filename)
    # df.to_excel(file_path, index=False, header=False)
 
def execute_scripts(driver, element=None, type=""):
    if type == "":
        print("Type not Correct")
        return driver
 
    REQ_SCRIPTS = {
        "color": """
            const style = document.createElement('style');
            style.type = 'text/css';
            style.innerHTML = `
                html, body {
                    background: white !important;
                    color: #111 !important;
                    -webkit-print-color-adjust: exact !important;
                    print-color-adjust: exact !important;
                }
                * {
                    color: #111 !important;
                    background: white !important;
                    border-color: #111 !important;
                }
                a, span, div, td, th, p, h1, h2, h3, h4, h5, h6 {
                    color: #111 !important;
                }
            `;
            document.head.appendChild(style);
        """
    }
 
    driverReturn = driver.execute_script(REQ_SCRIPTS[type], element) if element else driver.execute_script(REQ_SCRIPTS[type])
    time.sleep(1)
    return driverReturn
 
# --- Browser Setup ---
options = Options()
options.add_argument("start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36")
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)
driver.get(BASE_URL)
 
excelWriter = pd.ExcelWriter(XLS_FILE,engine="xlsxwriter")
 
WebDriverWait(driver, 5).until(EC.presence_of_element_located(SELECTORS["tab_content"]))
 
filter_list = driver.find_element(*SELECTORS["filter_list"])
last_all = filter_list.find_element(*SELECTORS["last_filter_li"])
last_all.click()
time.sleep(2)
 
pdf_links = driver.find_elements(*SELECTORS["fund_links"])
 
total_links = len(pdf_links)
print("Total links:", total_links)
 
for batch_start in range(0, total_links, BATCH_SIZE):
    batch = pdf_links[batch_start:batch_start + BATCH_SIZE]
    print(f"\nProcessing batch {(batch_start // BATCH_SIZE) + 1} with {len(batch)} links")
 
    for j, link in enumerate(batch):
        i = batch_start + j
        try:
            driver.execute_script("window.open(arguments[0].href, '_blank');", link)
            WebDriverWait(driver, 5).until(lambda d: len(d.window_handles) == 2)
            driver.switch_to.window(driver.window_handles[-1])
           
            try:
                performance_element = WebDriverWait(driver, 8).until(
                    EC.element_to_be_clickable(SELECTORS["portfolio_button"])
                )
                performance_element.click()
                WebDriverWait(driver, 8).until(lambda d: len(d.window_handles) == 3)
                driver.switch_to.window(driver.window_handles[-1])
                time.sleep(2)
            except:
               
                print("Portfolio click failed or not present")
           
            try:driver.find_element(*SELECTORS["show_more"]).click()
            except:driver.execute_script("document.getElementById('showMore1').click();")
 
            WebDriverWait(driver, 8).until(
                EC.presence_of_element_located(SELECTORS["companies_table"])
            )
 
            execute_scripts(driver, type="color")
       
            header = driver.find_element(*SELECTORS["page_header"]).text
            file_name, sfin = header.split("\n")
            content = {
                "url":driver.current_url,
                "name": clean_text(file_name),
                "sfin": sfin
            }
           
            print(f"[{i+1}] INSR NAME: {content["name"]} PDF URL: {content["url"]}")
 
            save_pdf(driver,file_name)
            save_table_to_excel(driver,content,excelWriter)
 
        except Exception as e:
            print(f"[FAIL] {file_name}")
            print(f"[{i+1}] Failed: {e}")
        finally:
            while len(driver.window_handles) > 1:
                driver.switch_to.window(driver.window_handles[-1])
                driver.close()
            driver.switch_to.window(driver.window_handles[0])
 
    if batch_start + BATCH_SIZE < total_links:
        print(f"Sleeping {COOLDOWN} seconds before next batch...")
        time.sleep(COOLDOWN)
       
excelWriter.close()
 

In [ ]:
import pandas as pd
from datetime import datetime

# Define start and end dates
start_date = datetime(2021, 1, 1)
end_date = datetime.today()

month_ends = pd.date_range(start=start_date, end=end_date, freq='M')
last_5_days = []
for date in month_ends:
    for i in range(4, -1, -1):  # Reverse to get earlier days first
        day = date - pd.Timedelta(days=i)
        formatted = day.strftime('%d-%b-%Y').lower()
        last_5_days.append(formatted)

last_5_days.sort(key=lambda x: datetime.strptime(x, '%d-%b-%Y'))
for d in last_5_days:
    print(d)

In [ ]:
import pandas as pd
import requests
from datetime import datetime

# dates = [
#     "27-sep-2025", "28-sep-2025", "29-sep-2025", "30-sep-2025", "01-oct-2025",
# ]
all_data = []

not_found = []

for dato in last_5_days:
    url = f"https://www.amfiindia.com/api/tracking-error-data?MF_ID=all&strdt={dato}"
    response = requests.get(url, verify=False)
    
    if response.status_code == 200:
        data = response.json()
        if not data["data"]:
            not_found.append(f"No Data Found for date: {dato}")
        df = pd.DataFrame(data["data"])
        df["date"] = dato 
        all_data.append(df)
    
    time.sleep(3)


with open("not_there.txt","w") as f:
    f.writelines(not_found)

final_df = pd.concat(all_data, ignore_index=True)
final_df.to_excel("MutualFundSchemes.xlsx", index=False)

In [4]:
import json
import re
import pandas as pd
from bs4 import BeautifulSoup

def parse_html_table(html):
    """Parse one HTML table into DataFrame with Date, Subject, Remarks, Link."""
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find("table")
    if not table:
        return None

    rows = []
    for tr in table.select("tbody tr"):
        cols = tr.find_all("td")
        if len(cols) < 3:
            print("There are less than three cols, skipping")
            continue

        date = cols[0].get_text(strip=True)
        remarks = cols[2].get_text(strip=True)

        a = cols[1].find("a")
        subject = a.get_text(" ", strip=True) if a else cols[1].get_text(strip=True)
        link = ""
        if a and a.get("onclick"):
            onclick = a["onclick"]
            if "newwindow1" in onclick:
                start = onclick.find("'") + 1
                end = onclick.rfind("'")
                link = onclick[start:end]

        rows.append([date, subject, remarks, link])

    return pd.DataFrame(rows, columns=["Date", "Subject", "Remarks", "Link"])

def clean_sheet_name(name, fallback="Sheet"):
    safe = re.sub(r'[\[\]\:\*\?\/\\]', "_", str(name))
    return safe[:31] if safe else fallback

def cache_to_excel(cache_file, excel_out="Orders_All.xlsx"):
    with open(cache_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    used_names = {}  # track counts per sheet name

    with pd.ExcelWriter(excel_out, engine="xlsxwriter") as writer:
        for scraped in data["records"][0]["scraped_data"]:
            if not scraped.get("data_present"):
                continue

            action = scraped.get("action", "unknown")
            webpage = scraped.get("webpage", "")
            for resp in scraped.get("response", []):
                if resp.get("type") != "table_html":
                    continue

                df = parse_html_table(resp["value"])
                if df is None or df.empty:
                    continue

                titles = resp.get("title", [])
                base_name = clean_sheet_name(titles[-1] if titles else action)

                count = used_names.get(base_name, 0) + 1
                used_names[base_name] = count
                sheet_name = base_name if count == 1 else clean_sheet_name(f"{base_name}_{count}")


                meta_rows = [["Action", action], ["Webpage", webpage]]
                for i, t in enumerate(titles, start=1):
                    meta_rows.append([f"Header{i}", t])
                meta = pd.DataFrame(meta_rows, columns=["Meta", "Value"])
                meta.to_excel(writer, sheet_name=sheet_name, index=False, startrow=0)

                df_visible = df[["Date", "Subject", "Remarks","Link"]]
                startrow = len(meta) + 2
                df_visible.to_excel(writer, sheet_name=sheet_name, index=False, startrow=startrow)

                # ws = writer.sheets[sheet_name]
                # for i, (txt, url) in enumerate(zip(df["Subject"], df["Link"]), start=startrow+1):
                #     if url:
                #         ws.write_url(i, 1, url, string=txt)

    print(f"Excel saved: {excel_out}")


# Example usage
path = r"C:\Users\kaustubh.keny\Projects\JSON25\scrape_output\cache\2025-09-23\25-09-23T15-32-09_cache.json"
cache_to_excel(path, "Orders_example_All.xlsx")


There are less than three cols, skipping
Excel saved: Orders_example_All.xlsx


In [ ]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
path = r"C:\Users\rando\Office Projects\outputs\scrape_output\cache\2025-09-23\25-09-23T22-31-06_cache.json"
data = Helper.load_json(path)
OperationExecutor.generate_cache_doc_report(data)

In [ ]:
import requests
import pandas as pd
from io import BytesIO,StringIO
from datetime import datetime

session = requests.Session()
session.headers.update({"User-Agent": "Mozilla/5.0"})

def fetch_rba(): 
    url = "https://www.rba.gov.au/statistics/tables/xls/f01d.xlsx" 
    r = session.get(url, timeout=15) 
    r.raise_for_status() 
    df = pd.read_excel(BytesIO(r.content)) 
    df.columns = df.iloc[1, :]
    df = df.iloc[-5:, :].T.reset_index()
    df.columns = df.iloc[0, :]
    df = df.drop(df.index[0])
    df.columns = [(lambda c: pd.to_datetime(c, errors="ignore"))(c) for c in df.columns]
    df.columns = [c.strftime("%m-%d-%Y") if isinstance(c, pd.Timestamp) else c for c in df.columns]
    
    return df

def fetch_bcra():
    url = "https://www.bcra.gob.ar/PublicacionesEstadisticas/Principales_variables_i.asp"
    r = session.get(url, timeout=15, verify=False)
    r.raise_for_status()
    df = pd.read_html(r.text)[0]
    
    today = datetime.today().strftime("%m/%d/%Y")
    date_range = pd.date_range(end=today, periods=5)   # datetime index

    df.columns = ["Header","Date","Value"]
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce", dayfirst=True)

    reshaped = (df.pivot_table(index="Header", columns="Date", values="Value", aggfunc="first"))
    reshaped = reshaped.reindex(columns=date_range)
    reshaped.columns = [d.strftime("%m-%d-%Y") for d in reshaped.columns]
    reshaped = reshaped.reset_index()
    return reshaped

def fetch_banxico(token, series_ids, start, end): 
    #mapper
    series_map = {
        "SF61745": "Target rate 1/",
        "SF331451": "Overnight TIIE funding rate 2/",
        "SF43783": "28 day TIIE 3/",
        "SF43878": "91 day TIIE 3/",
        "SF111916": "182 day TIIE 3/"
    }
    
    #api call
    url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{','.join(series_ids)}/datos/{start}/{end}"
    headers = {"Bmx-Token": token}
    r = session.get(url, headers=headers, timeout=15)
    r.raise_for_status()
    data = r.json()

    records = {}
    for s in data['bmx']['series']:
        sid = s['idSerie']
        rec = {d['fecha']: d['dato'] for d in s['datos']}
        records[sid] = rec
    df = pd.DataFrame.from_dict(records, orient="index")
    df.columns = pd.to_datetime(df.columns, dayfirst=True)
    
    df.index = df.index.map(lambda x: series_map.get(x, x))
    # print(df)
    df.columns = [d.strftime("%m-%d-%Y") for d in df.columns]
    
    return df.sort_index(axis=1)

def fetch_boc(series_ids, start, end):
    series_map = {
        "V39079": "Target for the overnight rate",
        "CL.CDN.MOST.1DL": "Overnight money market financing rate1",
        "V39078": "Bank rate - Daily",
        "V80691310": "Bank rate - Weekly",
        "V122530": "Bank rate - Monthly"
    }
    
    url = f"https://www.bankofcanada.ca/valet/observations/{','.join(series_ids)}/json?start_date={start}&end_date={end}"
    r = session.get(url, timeout=15)
    r.raise_for_status()
    data = r.json()

    records = {}
    for row in data["observations"]:
        date = row["d"]
        for sid in series_ids:
            records.setdefault(sid, {})[date] = row.get(sid, {}).get("v")

    df = pd.DataFrame.from_dict(records, orient="index")
    # make sure column labels are datetime
    df.columns = pd.to_datetime(df.columns)
    full_range = pd.date_range(start=start, end=end, freq="D")
    df = df.reindex(columns=full_range)
    df.columns = [d.strftime("%m-%d-%Y") for d in df.columns]
    df.index = df.index.map(lambda x: series_map.get(x, x))

    return df.sort_index(axis=1)


with pd.ExcelWriter("central_banks_1.xlsx") as writer:
    # token = "1846d489e18513d4306b70e257a21edb56e82325de0139af91dd2176975029bf"
    # fetch_banxico(token, ["SF61745","SF331451","SF43783","SF43878","SF111916"], "2025-09-25", "2025-09-29")\
    #     .to_excel(writer, sheet_name="Banxico")
    # fetch_boc(["V39079","CL.CDN.MOST.1DL","V39078","V80691310","V122530"], "2025-09-25", "2025-09-29")\
    #     .to_excel(writer, sheet_name="BoC")
    # fetch_rba().to_excel(writer, sheet_name="RBA", index = False)
    fetch_bcra().to_excel(writer, sheet_name="BCRA", index=False)

In [1]:
import requests, re, shutil, os
import pandas as pd
from io import BytesIO
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from openpyxl import load_workbook
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

session = requests.Session()
session.headers.update({"User-Agent": "Mozilla/5.0"})


# ---- Shared Pandas Helpers ----
def normalize_columns_to_dates(df):
    """Convert datetime-like columns to mm-dd-YYYY strings."""
    df.columns = [
        c.strftime("%m-%d-%Y") if isinstance(c, pd.Timestamp) else c
        for c in df.columns
    ]
    return df

def ensure_full_date_range(df, start, end):
    """Ensure dataframe has full date coverage, fill missing dates with NaN."""
    full_range = pd.date_range(start=start, end=end, freq="D")
    df = df.reindex(columns=full_range)
    return normalize_columns_to_dates(df)

def format_header(header):
    if isinstance(header, (pd.Timestamp, datetime)):
        return header.strftime("%d%m%Y")
    if not isinstance(header, str):
        return str(header)

    for fmt in ("%Y-%m-%d", "%Y-%m-%d %H:%M:%S", "%y%m%d", "%m/%d/%Y", "%d-%m-%Y", "%d/%m/%Y", "%m-%d-%Y"):
        try:
            parsed = datetime.strptime(header, fmt)
            return parsed.strftime("%d%m%Y")
        except ValueError:
            continue
    return header
    return header
# ---- Fetchers ----
def fetch_rba(cfg, start=None, end=None):
    r = session.get(cfg["url"], timeout=15)
    r.raise_for_status()
    df = pd.read_excel(BytesIO(r.content))

    df.columns = df.iloc[1, :]
    df = df.iloc[-5:, :].T.reset_index()
    df.columns = df.iloc[0, :]
    df = df.drop(df.index[0])

    return normalize_columns_to_dates(df)

def fetch_bcra(cfg, start=None, end=None):
    r = session.get(cfg["url"], timeout=15, verify=False)
    r.raise_for_status()
    df = pd.read_html(r.text)[0]

    today = datetime.today().strftime("%m/%d/%Y")
    date_range = pd.date_range(end=today, periods=5)   # last 5 days

    df.columns = ["Header", "Date", "Value"]
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce", dayfirst=True)

    reshaped = df.pivot_table(index="Header", columns="Date", values="Value", aggfunc="first")
    reshaped = reshaped.reindex(columns=date_range)
    reshaped = normalize_columns_to_dates(reshaped)
    return reshaped.reset_index()

def fetch_banxico(cfg,start, end): 

    series_map = cfg.get("series_map")
    series_ids = cfg.get("series_ids")
    headers =  cfg.get("headers")
    base_url =  cfg.get("url")
    #api call
    url = f"{base_url}/{','.join(series_ids)}/datos/{start}/{end}"
    r = session.get(url, headers=headers, timeout=15)
    r.raise_for_status()
    data = r.json()

    records = {}
    for s in data['bmx']['series']:
        sid = s['idSerie']
        rec = {d['fecha']: d['dato'] for d in s['datos']}
        records[sid] = rec
    df = pd.DataFrame.from_dict(records, orient="index")
    df.columns = pd.to_datetime(df.columns, dayfirst=True)
    
    df.index = df.index.map(lambda x: series_map.get(x, x))
    df.columns = [d.strftime("%m-%d-%Y") for d in df.columns]
    return df.sort_index(axis=1)

def fetch_boc(cfg, start, end):
    series_map = cfg.get("series_map")
    series_ids = cfg.get("series_ids")
    base_url =  cfg.get("url")
    
    url = f"{base_url}/{','.join(series_ids)}/json?start_date={start}&end_date={end}"
    r = session.get(url, timeout=15)
    r.raise_for_status()
    data = r.json()

    records = {}
    for row in data["observations"]:
        date = row["d"]
        for sid in series_ids:
            records.setdefault(sid, {})[date] = row.get(sid, {}).get("v")

    df = pd.DataFrame.from_dict(records, orient="index")
    # make sure column labels are datetime
    df.columns = pd.to_datetime(df.columns)
    full_range = pd.date_range(start=start, end=end, freq="D")
    df = df.reindex(columns=full_range)
    df.columns = [d.strftime("%m-%d-%Y") for d in df.columns]
    df.index = df.index.map(lambda x: series_map.get(x, x))

    return df.sort_index(axis=1)

def fetch_boj(cfg, start, end):
    dates = pd.date_range(start=datetime.strptime(start, "%y%m%d"), end=datetime.strptime(end, "%y%m%d"))
    records = {}

    for date in dates:
        url = f"{cfg['url']}/md{date.strftime('%y%m%d')}.htm"
        try:
            r = session.get(url, timeout=15)
            if r.status_code != 200:continue
        except Exception:continue

        soup = BeautifulSoup(r.text, "html.parser")
        for li in soup.find_all("li"):
            text = li.get_text(strip=True)
            text = re.sub(r"[\[\%]+","",text,re.IGNORECASE)
            parts = text.split("]")
            if len(parts) < 2:continue
            key,val = parts
            # print(f"{key}:{val}")
            series_name = f"Call Rate {key}"
            records.setdefault(series_name, {})[date] = val

    out = pd.DataFrame.from_dict(records, orient="index")
    out = out.reindex(columns=dates)
    return normalize_columns_to_dates(out)

def fetch_nyfed(cfg, start, end):
    url = (
        f"{cfg['url']}?startDt={start}&endDt={end}"
        f"&sort={cfg['headers']['sort']}"
        f"&productCode={cfg['headers']['productCode']}"
        f"&eventCodes={cfg['headers']['eventCodes']}"
        f"&format={cfg['headers']['format']}"
    )
    r = session.get(url, timeout=15)
    r.raise_for_status()

    if cfg["headers"]["format"] == "csv":
        df = pd.read_csv(BytesIO(r.content))
    else:
        df = pd.read_excel(BytesIO(r.content))

    df.columns = [str(c).strip() for c in df.columns]
    return df

def fetch_cboe(cfg, start=None, end=None, days=5):
    r = session.get(cfg["url"], timeout=15, verify=False)
    r.raise_for_status()

    df = pd.read_csv(BytesIO(r.content))
    df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce")
    df = df.iloc[-days:, :]
    start_dt = pd.to_datetime(start)
    end_dt = pd.to_datetime(end)
    full_range = pd.date_range(start=start_dt, end=end_dt, freq="D")
    df = df.set_index("DATE").reindex(full_range)
    wide = pd.DataFrame([df["OVX"].tolist()], index=["OVX Index"], columns=full_range)
    return normalize_columns_to_dates(wide)

CONFIG = {
    "Banxico": {
        "fetcher": fetch_banxico,
        "url": "https://www.banxico.org.mx/SieAPIRest/service/v1/series",
        "headers": {
            "Bmx-Token": "1846d489e18513d4306b70e257a21edb56e82325de0139af91dd2176975029bf"
        },
        "series_ids":["SF61745","SF331451","SF43783","SF43878","SF111916"],
        "series_map": {
            "SF61745": "Target rate 1/",
            "SF331451": "Overnight TIIE funding rate 2/",
            "SF43783": "28 day TIIE 3/",
            "SF43878": "91 day TIIE 3/",
            "SF111916": "182 day TIIE 3/"
        }
    },
    "BoC": {
        "fetcher": fetch_boc,
        "url": "https://www.bankofcanada.ca/valet/observations",
        "series_ids": ["V39079","CL.CDN.MOST.1DL","V39078","V80691310","V122530"],
        "series_map": {
            "V39079": "Target for the overnight rate",
            "CL.CDN.MOST.1DL": "Overnight money market financing rate1",
            "V39078": "Bank rate - Daily",
            "V80691310": "Bank rate - Weekly",
            "V122530": "Bank rate - Monthly"
        }
    },
    "RBA": {
        "fetcher": fetch_rba,
        "url": "https://www.rba.gov.au/statistics/tables/xls/f01d.xlsx"
    },
    "BCRA": {
        "fetcher": fetch_bcra,
        "url": "https://www.bcra.gob.ar/PublicacionesEstadisticas/Principales_variables_i.asp"
    },
    "BOJ": {
        "fetcher": fetch_boj,
        "url": "https://www3.boj.or.jp/market/en/stat"
    },
    "NYCFED": {
        "fetcher": fetch_nyfed,
        "url": "https://markets.newyorkfed.org/read",
        "headers": {
            "sort": "postDt:-1,'data.closeTm':-1",
            "productCode": 70,     # repo operations
            "eventCodes": 730,     # event code for operations
            "format": "csv"        # NY Fed supports csv or xls
        }
    },
    "CBOE": {
        "fetcher": fetch_cboe,
        "url": "https://cdn.cboe.com/api/global/us_indices/daily_prices/OVX_History.csv"
    }
}


def run_pipeline(days=5, outfile="Daily Interest Rate.xlsx"):
    end = datetime.today()
    start = end - timedelta(days=days)

    with pd.ExcelWriter(outfile) as writer:
        for name, cfg in CONFIG.items():
            fetcher = cfg["fetcher"]
            try:
                if name in ["Banxico"]:
                    df = fetcher(cfg, start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"))
                elif name in ["BoC"]:
                    df = fetcher(cfg, start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"))
                elif name in ["BOJ"]:
                    df = fetcher(cfg, start.strftime("%y%m%d"), end.strftime("%y%m%d"))
                elif name in ["NYCFED"]:
                    df = fetcher(cfg, start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"))
                elif name in ["CBOE"]:
                    df = fetcher(cfg, start.strftime("%m/%d/%Y"), end.strftime("%m/%d/%Y"))
                else:
                    df = fetcher(cfg, start, end)
                if df is not None and not df.empty:
                    df.columns = [format_header(d) for d in df.columns]
                    df.to_excel(writer, sheet_name=name, index=not name in ["RBA", "BCRA","NYCFED"])
                else:
                    print(f"{name}: No data returned")

            except Exception as e:
                print(f"Error fetching {name}: {e}")

    print(f"Data saved to {outfile}")

# def run_pipeline(days=5, outfile="Daily Interest Rate.xlsx"):
#     end = datetime.today()
#     start = end - timedelta(days=days)

#     # Step 1: Backup the existing file
#     if os.path.exists(outfile):
#         timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
#         backup_file = outfile.replace(".xlsx", f"_backup_{timestamp}.xlsx")
#         shutil.copy2(outfile, backup_file)
#         print(f"Backup created: {backup_file}")

#     # Step 2: Update sheets using ExcelWriter in append mode
#     for name, cfg in CONFIG.items():
#         fetcher = cfg["fetcher"]
#         try:
#             if name in ["Banxico", "BoC", "NYCFED"]:
#                 df = fetcher(cfg, start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"))
#             elif name == "BOJ":
#                 df = fetcher(cfg, start.strftime("%y%m%d"), end.strftime("%y%m%d"))
#             elif name == "CBOE":
#                 df = fetcher(cfg, start.strftime("%m/%d/%Y"), end.strftime("%m/%d/%Y"))
#             else:
#                 df = fetcher(cfg, start, end)

#             if df is not None and not df.empty:
#                 df.columns = [format_header(d) for d in df.columns]

#                 # Remove the sheet if it exists
#                 book = load_workbook(outfile)
#                 if name in book.sheetnames:
#                     std = book[name]
#                     book.remove(std)
#                     book.save(outfile)

#                 # Write updated sheet
#                 with pd.ExcelWriter(outfile, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
#                     df.to_excel(writer, sheet_name=name, index=not name in ["RBA", "BCRA", "NYCFED"])
#             else:
#                 print(f"{name}: No data returned")

#         except Exception as e:
#             print(f"Error fetching {name}: {e}")

#     print(f"Data updated in {outfile}")

if __name__ == "__main__":
    run_pipeline(days=5)

C:\Users\kaustubh.keny\AppData\Local\Temp\ipykernel_21004\3839537942.py:59: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(r.text)[0]


Data saved to Daily Interest Rate.xlsx


In [ ]:
url = r"https://markets.newyorkfed.org/read?startDt=2025-09-23&endDt=2025-09-27&sort=postDt:-1,'data.closeTm':-1&productCode=70&eventCodes=730&format=csv"

In [3]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
from urllib.parse import urljoin
import os
ops = OperationExecutor()
path = r"C:\Users\kaustubh.keny\Projects\JSON25\scrape_output\cache\2025-09-26\25-09-26T15-44-07_cache.json"
# path = r"25-09-08T18-33-09_cache.json5"
path = r"C:\Users\kaustubh.keny\Projects\JSON25\scrape_output\cache\2025-09-26\25-09-26T15-39-53_cache.json"

    
root_file = os.path.basename(path).split("_")[0]
function_to_execute = {
"primary":[["normalize_df","value","norm_table","table_html"],["sha1","value","SHA_ONE","pdf"]],
"secondary":[["sha1","norm_table","SHA_ONE"]],
}
try:
    data = Helper.load_json(path, typ="json5")
    processed_data = ops.runner(data,function_to_execute)
    Helper.save_json(processed_data,f"{root_file}_process.json")
except Exception as e:
    print(f"\nError in Processing.. Skipping, {e}")


>>Processing Bank of Baroda


In [4]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
ops = OperationExecutor()

v1 = r"25-09-26T15-44-07_process.json"
v2 = r"25-09-26T15-39-53_process.json"
yesterday = Helper.load_json(v2)
today = Helper.load_json(v1)

result = ops.process_comparison(yesterday,today,key="SHA_ONE")
Helper.save_json(result, "compare-080925.json")

ops.generate_sorted_excel_report(result,output_path="Despoit_Rate_Report.xlsx")

'Despoit_Rate_Report.xlsx'

In [ ]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
ops = OperationExecutor()

path = r"C:\Users\kaustubh.keny\Projects\JSON25\scrape_output\cache\1109_1009_compare\compare-080925.json"
result = Helper.load_json(path)
ops.generate_sorted_excel_report(result,output_path="Despoit_Rate_Report.xlsx")

'Despoit_Rate_Report.xlsx'